In [8]:
root_dir="/checkpoint/tomsander/data"
import sys
sys.path.insert(0, '/private/home/tomsander/vision-language-models-are-bows')


In [ ]:
from dataset_zoo import VG_Relation, VG_Attribution
import torch
vgr_dataset = VG_Relation(image_preprocess=None, download=True, root_dir=root_dir)

In [4]:
image = vgr_dataset[0]['image_options'][0]

In [5]:
import torchvision.transforms as transforms
transform = transforms.Compose([
    transforms.PILToTensor()
])
img_tensor = transform(image)
print(img_tensor)

tensor([[[111, 110, 109,  ..., 177, 171, 187],
         [116, 114, 114,  ..., 175, 178, 175],
         [123, 121, 118,  ..., 170, 173, 170],
         ...,
         [  5,   6,   5,  ...,   8,  10,   7],
         [  4,   3,   5,  ...,   7,  12,   9],
         [  4,   5,   6,  ...,   8,   7,   7]],

        [[104, 103, 102,  ..., 177, 173, 188],
         [108, 106, 106,  ..., 178, 183, 179],
         [112, 110, 107,  ..., 173, 179, 176],
         ...,
         [  5,   7,   5,  ...,  10,  12,   9],
         [  5,   4,   6,  ...,   9,  14,  11],
         [  4,   6,   6,  ...,  10,   9,   9]],

        [[ 85,  84,  83,  ..., 187, 170, 190],
         [ 89,  87,  87,  ..., 187, 179, 180],
         [ 92,  90,  87,  ..., 182, 175, 176],
         ...,
         [  3,   2,   3,  ...,   9,  11,   8],
         [  0,   0,   1,  ...,   6,  11,   8],
         [  2,   1,   4,  ...,   7,   6,   6]]], dtype=torch.uint8)


In [7]:
from PIL import Image
image_preprocess = transforms.Compose([
                    transforms.Resize((384, 384), interpolation=Image.BICUBIC),
                    transforms.ToTensor(),
                    transforms.Normalize((0.48145466, 0.4578275, 0.40821073), (0.26862954, 0.26130258, 0.27577711)),])

In [ ]:
device = torch.device('cuda')
#model
model = models_mae.__dict__[args.model](norm_pix_loss=False, text_max_length=args.target_txt_len)
model.to(device)
model.eval()
checkpoint = torch.load(args.checkpoint_path, map_location='cpu') #why cpu?
model.load_state_dict(checkpoint['model'])
#tokinizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
#data
transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224,224)),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
if args.dataset=="ImageNet":
    dataset_train = datasets.ImageFolder(os.path.join(args.data_path, args.partition), transform=transform_test)
    sampler_train = torch.utils.data.RandomSampler(dataset_train, num_samples=args.nb_eval)
    lengths, dic2, mask_ = labels_lengths_with_BOS(tokenizer, "labels/imagenet_labels.json", dataset=args.dataset)
elif args.dataset=="CIFAR10":
    dataset_train = torchvision.datasets.CIFAR10(root='./data', train=True,download=True, transform=transform_test)
    sampler_train = torch.utils.data.RandomSampler(dataset_train, num_samples=args.nb_eval)
    lengths, dic2, mask_ = labels_lengths_with_BOS(tokenizer, "labels/cifar10_labels.json", dataset=args.dataset)
elif args.dataset=="CIFAR100":
    dataset_train = torchvision.datasets.CIFAR100(root='./data', train=True,download=True, transform=transform_test)
    sampler_train = torch.utils.data.RandomSampler(dataset_train, num_samples=args.nb_eval)
    lengths, dic2, mask_ = labels_lengths_with_BOS(tokenizer, "labels/cifar100_labels.json", dataset=args.dataset)
data_loader_train = torch.utils.data.DataLoader(
    dataset_train, sampler=sampler_train,
    batch_size=1,
    num_workers=10,
    pin_memory="store_true",
    drop_last=True,
)
proportions = {'1': 0, '2': 0}

good=0
count = 0
text = ['this is a photo of a ']
texts_tokenized_input = tokenizer.batch_encode_plus(text,add_special_tokens=True,return_tensors='pt')
input_ids_ = texts_tokenized_input['input_ids'].to(device)
input_ids_ = input_ids_[0][:-1].unsqueeze(0)
for (image,label) in tqdm(data_loader_train):
    res = -1
    loss_res = 1000000
    image = image.squeeze(0).to(device)
    latent, _, _, _, _ = model.forward_encoder(image.unsqueeze(0), 0)
    # decoder
    x = model.decoder_embed(latent)
    #Adding Image Posiotion embedding
    x = x + model.decoder_pos_embed
    x = x + model.decoder_image_type_embedding
    key_padding_mask=torch.zeros(x.shape[0],x.shape[1],dtype=torch.bool, device=device)#useless
    # x = rearrange(x, 'b n c->n b c') 
    for key, values in dic2.items():
        for s in values:
            #texts = torch.Tensor([s]).to(device).long()
            texts = torch.Tensor([input_ids_[0].tolist()+s[1:]]).to(device).long()
            t = model.text_embedding_layer_SimVLM(texts)
            t = t + model.text_decoder_pos_embed[:, :t.shape[1], :]
            t = t + model.decoder_text_type_embedding
            target_text_length=len(texts[0])
            tgt_mask = generate_square_subsequent_mask(target_text_length).to(device)
            tgt_key_padding_mask=texts==0#shouldnt be used bause we do not use padding tokens
            # t = rearrange(t, 'b n c->n b c')
            output = model.decoder(t, x, is_causal=1, 
                                    tgt_mask=tgt_mask, memory_mask=None,
                                    tgt_key_padding_mask=tgt_key_padding_mask,memory_key_padding_mask=key_padding_mask)
            output = model.decoder_pred_text(output)
            # output = rearrange(output, 'n b c -> b c n')
            output = rearrange(output, 'b n c -> b c n')
            target_text = texts[:,1:]
            loss=F.cross_entropy(output[:,:,:-1],target_text)
            if loss.item() < loss_res:
                loss_res = loss.item()
                res = int(key)
    if res ==label.item():
        good+=1
    count+=1
print(good/count)